# Day 16: Packet Decoder

https://adventofcode.com/2021/day/16

In [1]:
def get_packet(txt):
    """Convert the hexadecimal representation into binary."""
    packet = bin(int('0x' + txt, base=16))[2:].zfill(len(txt)*4)
    return packet

In [2]:
def parse_packet(packet, version_sum=0):
    """Parse a packet, calculate the version sum and apply rules."""
    packet_version = int(packet[0:3], base=2)
    version_sum += packet_version
    packet_type = int(packet[3:6], base=2)
    if packet_type == 4:  # literal value
        groups = [packet[6:][i*5:i*5+5] for i in range(1+len(packet[6:])//5)]
        number = ''
        for i, group in enumerate(groups):
            number += group[1:5]
            if group[0] == '0':
                packet_length = 6 + i*5+5
                packet = packet[packet_length:]  # remove processed part of packet
                break  # last group
        number = int(number, base=2)
    else:  # operator
        length_type_id = packet[6]
        numbers = []  # store literal values of sub-packets
        if length_type_id == '0':
            # If the length type ID is 0, then the next 15 bits are a number that represents
            # the total length in bits of the sub-packets contained by this packet.
            subpackets_length = int(packet[7:7+15], base=2)
            packet = packet[7+15:]  # remove processed part of packet
            length_subpackets = 0
            while length_subpackets < subpackets_length:
                length_subpackets += len(packet)
                packet, version_sum, number = parse_packet(packet, version_sum)  # call recursively
                length_subpackets -= len(packet)
                numbers.append(number)
        else:
            # If the length type ID is 1, then the next 11 bits are a number that represents
            # the number of sub-packets immediately contained by this packet.
            subpackets_number = int(packet[7:7+11], base=2)
            packet = packet[7+11:]  # remove processed part of packet
            number_subpackets = 0
            while number_subpackets < subpackets_number:
                packet, version_sum, number = parse_packet(packet, version_sum)  # call recursively
                number_subpackets += 1
                numbers.append(number)
        number = apply_rules(packet_type, numbers)  # apply rule to literal values of sub-packets
    return packet, version_sum, number

In [3]:
def apply_rules(packet_type, numbers):
    """Apply the relevant rule for the packet type ID."""
    if packet_type == 0:  # sum packet
        number = sum(numbers)
    elif packet_type == 1:  # product packet
        number = 1
        for n in numbers:
            number *= n
    elif packet_type == 2:  # minimum packet
        number = min(numbers)
    elif packet_type == 3:  # maximum packet
        number = max(numbers)
    elif packet_type == 5:  # greater than packet
        number = int(numbers[0] > numbers[1])
    elif packet_type == 6:  # less than packet
        number = int(numbers[0] < numbers[1])
    elif packet_type == 7:  # equal to packet
        number = int(numbers[0] == numbers[1])
    return number

In [4]:
part1_examples = ['D2FE28', '38006F45291200', 'EE00D40C823060', '8A004A801A8002F478',
                  '620080001611562C8802118E34', 'C0015000016115A2E0802F182340',
                  'A0016C880162017C3686B18A3D4780']

In [5]:
part2_examples = ['C200B40A82', '04005AC33890', '880086C3E88112', 'CE00C43D881120', 'D8005AC2A8F0',
                  'F600BC2D8F', '9C005AC2F8F0', '9C0141080250320F1802104A08']

In [6]:
with open('input.txt') as input_file:
    input_txt = input_file.read().strip()

Check the code gives the correct answers for the examples then run it on the input text.

In [7]:
for txt in part1_examples + part2_examples + [input_txt]:
    print(txt)
    packet = get_packet(txt)
    remaining_packet, version_sum, number = parse_packet(packet)
    print(f'Version sum is {version_sum} and answer is {number}.\n')

D2FE28
Version sum is 6 and answer is 2021.

38006F45291200
Version sum is 9 and answer is 1.

EE00D40C823060
Version sum is 14 and answer is 3.

8A004A801A8002F478
Version sum is 16 and answer is 15.

620080001611562C8802118E34
Version sum is 12 and answer is 46.

C0015000016115A2E0802F182340
Version sum is 23 and answer is 46.

A0016C880162017C3686B18A3D4780
Version sum is 31 and answer is 54.

C200B40A82
Version sum is 14 and answer is 3.

04005AC33890
Version sum is 8 and answer is 54.

880086C3E88112
Version sum is 15 and answer is 7.

CE00C43D881120
Version sum is 11 and answer is 9.

D8005AC2A8F0
Version sum is 13 and answer is 1.

F600BC2D8F
Version sum is 19 and answer is 0.

9C005AC2F8F0
Version sum is 16 and answer is 0.

9C0141080250320F1802104A08
Version sum is 20 and answer is 1.

220D4B80491FE6FBDCDA61F23F1D9B763004A7C128012F9DA88CE27B000B30F4804D49CD515380352100763DC5E8EC000844338B10B667A1E60094B7BE8D600ACE774DF39DD364979F67A9AC0D1802B2A41401354F6BF1DC0627B15EC5CCC01694